# Byte-at-a-time ECB decryption (Simple)
Copy your oracle function to a new function that encrypts buffers under ECB mode using a consistent but unknown key (for instance, assign a single random key, once, to a global variable).

Now take that same function and have it append to the plaintext, BEFORE ENCRYPTING, the following string:
```
Um9sbGluJyBpbiBteSA1LjAKV2l0aCBteSByYWctdG9wIGRvd24gc28gbXkg
aGFpciBjYW4gYmxvdwpUaGUgZ2lybGllcyBvbiBzdGFuZGJ5IHdhdmluZyBq
dXN0IHRvIHNheSBoaQpEaWQgeW91IHN0b3A/IE5vLCBJIGp1c3QgZHJvdmUg
YnkK
```
### Spoiler alert.
> Do not decode this string now. Don't do it.

Base64 decode the string before appending it. Do not base64 decode the string by hand; make your code do it. The point is that you don't know its contents.

What you have now is a function that produces:
```
AES-128-ECB(your-string || unknown-string, random-key)
```
It turns out: you can decrypt "unknown-string" with repeated calls to the oracle function!

Here's roughly how:

1. Feed identical bytes of your-string to the function 1 at a time --- start with 1 byte ("A"), then "AA", then "AAA" and so on. Discover the block size of the cipher. You know it, but do this step anyway.
2. Detect that the function is using ECB. You already know, but do this step anyways.
3. Knowing the block size, craft an input block that is exactly 1 byte short (for instance, if the block size is 8 bytes, make "AAAAAAA"). Think about what the oracle function is going to put in that last byte position.
4. Make a dictionary of every possible last byte by feeding different strings to the oracle; for instance, "AAAAAAAA", "AAAAAAAB", "AAAAAAAC", remembering the first block of each invocation.
5. Match the output of the one-byte-short input to one of the entries in your dictionary. You've now discovered the first byte of unknown-string.
6. Repeat for the next byte.

### Congratulations.
> This is the first challenge we've given you whose solution will break real crypto. Lots of people know that when you encrypt something in ECB mode, you can see penguins through it. Not so many of them can decrypt the contents of those ciphertexts, and now you can. If our experience is any guideline, this attack will get you code execution in security tests about once a year.

In [1]:
from cryptopals import *

In [6]:
def appending_ECB_oracle(plaintext, key):
    '''Oracle that appends a mystery string to the plaintext input before encrypting via ECB.'''
    unknown_string = base64_to_bytes('Um9sbGluJyBpbiBteSA1LjAKV2l0aCBteSByYWctdG9wIGRvd24gc28gbXkgaGFpciBjYW4gYmxvdwpUaGUgZ2lybGllcyBvbiBzdGFuZGJ5IHdhdmluZyBqdXN0IHRvIHNheSBoaQpEaWQgeW91IHN0b3A/IE5vLCBJIGp1c3QgZHJvdmUgYnkK')
    plaintext = pad_pkcs7(to_bytes(plaintext) + unknown_string)
    
    return AES.new(key, AES.MODE_ECB).encrypt(plaintext)

def find_unknown_string_from_appending_oracle():
    # generate consistent but unknown key
    consistent_unknown_key = generate_random_bytes()
    
    # find block size
    BLOCK_SIZE = 1
    while True:
        ct = appending_ECB_oracle('A'*2*BLOCK_SIZE, consistent_unknown_key)
        if ct[0:BLOCK_SIZE] == ct[BLOCK_SIZE:2*BLOCK_SIZE]:
            break
        BLOCK_SIZE += 1
    assert(BLOCK_SIZE == 16)
        
    # ensure the cipher is ECB
    assert(detect_ECB_CBC_oracle(lambda x: appending_ECB_oracle(x, consistent_unknown_key)) == 'ECB')
    
    # find unknown message, one byte at a time
    hidden_string = ''
    
    while True:
        idx_next_letter = len(hidden_string)
        idx_start_of_block = idx_next_letter - idx_next_letter%BLOCK_SIZE
        short_pt = chr(0)*(BLOCK_SIZE-1-idx_next_letter%BLOCK_SIZE)

        ct_block_to_char_map = {}
        for i in range(256):
            pt = short_pt + hidden_string + chr(i)
            ct = appending_ECB_oracle(pt, consistent_unknown_key)
            unknown_block = ct[idx_start_of_block:idx_start_of_block+BLOCK_SIZE]
            ct_block_to_char_map[unknown_block] = chr(i)

        ct = appending_ECB_oracle(short_pt, consistent_unknown_key)
        unknown_block = ct[idx_start_of_block:idx_start_of_block+BLOCK_SIZE]
        if unknown_block not in ct_block_to_char_map:
            # signals end of message because padding bytes don't match any possible
            return hidden_string[:-1]
        hidden_string += ct_block_to_char_map[unknown_block]

In [3]:
consistent_unknown_key = generate_random_bytes()

In [7]:
BLOCK_SIZE = 1
while True:
    ct = appending_ECB_oracle('A'*2*BLOCK_SIZE, consistent_unknown_key)
    if ct[0:BLOCK_SIZE] == ct[BLOCK_SIZE:2*BLOCK_SIZE]:
        print("Block size is %d" % BLOCK_SIZE)
        break
    BLOCK_SIZE += 1

Block size is 16


In [8]:
detect_ECB_CBC_oracle(lambda x: appending_ECB_oracle(x, consistent_unknown_key))

'ECB'

In [9]:
hidden_string = ''

In [10]:
short_pt = chr(0)*(BLOCK_SIZE-1)

first_ct_block_to_char_map = {}
for i in range(256):
    ct = appending_ECB_oracle(short_pt+chr(i), consistent_unknown_key)
    first_ct_block_to_char_map[ct[:BLOCK_SIZE]] = chr(i)

In [11]:
ct = appending_ECB_oracle(short_pt, consistent_unknown_key)
hidden_string += first_ct_block_to_char_map[ct[:BLOCK_SIZE]]
print(hidden_string)

R


In [13]:
%%time
while True:
    idx_next_letter = len(hidden_string)
    idx_start_of_block = idx_next_letter - idx_next_letter%BLOCK_SIZE
    short_pt = chr(0)*(BLOCK_SIZE-1-idx_next_letter%BLOCK_SIZE)

    ct_block_to_char_map = {}
    for i in range(256):
        pt = short_pt + hidden_string + chr(i)
        ct = appending_ECB_oracle(pt, consistent_unknown_key)
        unknown_block = ct[idx_start_of_block:idx_start_of_block+BLOCK_SIZE]
        ct_block_to_char_map[unknown_block] = chr(i)

    ct = appending_ECB_oracle(short_pt, consistent_unknown_key)
    unknown_block = ct[idx_start_of_block:idx_start_of_block+BLOCK_SIZE]
    if unknown_block not in ct_block_to_char_map:
        # signals end of message because padding bytes don't match any possible
        break
    hidden_string += ct_block_to_char_map[unknown_block]
    
    print(hidden_string)

Ro
Rol
Roll
Rolli
Rollin
Rollin'
Rollin' 
Rollin' i
Rollin' in
Rollin' in 
Rollin' in m
Rollin' in my
Rollin' in my 
Rollin' in my 5
Rollin' in my 5.
Rollin' in my 5.0
Rollin' in my 5.0

Rollin' in my 5.0
W
Rollin' in my 5.0
Wi
Rollin' in my 5.0
Wit
Rollin' in my 5.0
With
Rollin' in my 5.0
With 
Rollin' in my 5.0
With m
Rollin' in my 5.0
With my
Rollin' in my 5.0
With my 
Rollin' in my 5.0
With my r
Rollin' in my 5.0
With my ra
Rollin' in my 5.0
With my rag
Rollin' in my 5.0
With my rag-
Rollin' in my 5.0
With my rag-t
Rollin' in my 5.0
With my rag-to
Rollin' in my 5.0
With my rag-top
Rollin' in my 5.0
With my rag-top 
Rollin' in my 5.0
With my rag-top d
Rollin' in my 5.0
With my rag-top do
Rollin' in my 5.0
With my rag-top dow
Rollin' in my 5.0
With my rag-top down
Rollin' in my 5.0
With my rag-top down 
Rollin' in my 5.0
With my rag-top down s
Rollin' in my 5.0
With my rag-top down so
Rollin' in my 5.0
With my rag-top down so 
Rollin' in my 5.0
With my rag-top down so m
Rollin' in my

In [15]:
%%time
print(find_unknown_string_from_appending_oracle())

Rollin' in my 5.0
With my rag-top down so my hair can blow
The girlies on standby waving just to say hi
Did you stop? No, I just drove by

CPU times: user 396 ms, sys: 7.82 ms, total: 404 ms
Wall time: 414 ms
